In [117]:
!pip install xlsxwriter

     |████████████████████████████████| 149 kB 9.9 MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import sys
sys.path.append('/content/drive/MyDrive/AI-Future/LocalSensitiveHashing/kdtree.py')
sys.path.append('/content/drive/MyDrive/AI-Future/LocalSensitiveHashing/lsh.py')
%cd "/content/drive/MyDrive/AI-Future/LocalSensitiveHashing"
%pwd

/content/drive/MyDrive/AI-Future/LocalSensitiveHashing


'/content/drive/MyDrive/AI-Future/LocalSensitiveHashing'

In [20]:
import kdtree
import lsh

In [118]:
import pandas as pd
import time
import numpy as np
import random
import xlsxwriter

#gen_data


In [23]:
n = [10,100,1000,10000,100000,1000000]
bin = [64,128,256,512,1024,8192,32768,131072,1048576]

In [24]:
ele = n[3]
filename = "a.txt"
random.seed(0)
lsh.gen_data(ele,"a.txt")

#CreateModule


In [25]:
method_tree = ['classic','re_balance']
method_lsh = ['universe','flipped']

In [81]:
def gen_query_point(l_gen,name,method = None):
    if name == "tree":
       if method == "classic" or method == "re_balance":
          point = kdtree.Point()
          point.set_x(round(float(l_gen[0]),3))
          point.set_y(round(float(l_gen[1]),3))
          point.set_z(round(float(l_gen[2]),3))
       else:
          point = [round(float(x),3) for x in l_gen]
    else:
      point = [round(float(x),3) - 50.000 for x in l_gen]
      point = np.array(point).reshape(1,3)
    return point

k = 3
l_gen = []
for j in range(0,3):
    g = round(random.uniform(0,100),3)
    l_gen.append(g)

#construct_method

In [59]:
model_trees = []
point_trees = []
time_construct = []
for method in method_tree:
    start = time.time()
    model,point = kdtree.build_method("a.txt",method)
    end = time.time()

    model_trees.append(model)
    point_trees.append(point)
    time_construct.append(end - start)

start = time.time()
points_hash = lsh.readData('a.txt')
model_hash = lsh.LSH(points_hash)
model_hash.create_planes(10,25)
model_hash.create_hash_tables()
end = time.time()
time_construct.append(end - start)

In [93]:
time_construct
time_construct.append(time_construct[-1])

#search_method


In [61]:
index_random = random.sample(range(len(points_hash)),1)[0]
time_search = []
for model,points in zip(model_trees,point_trees):
    #search point in data
    point = points[index_random]
    start = time.time()
    kdtree.search(point,model)
    end = time.time()
    time_search.append(end - start)

for method in method_lsh:
    start = time.time()
    id = model_hash.check_point_in_data(points_hash[index_random])
    end = time.time()
    if id == -1:
       print("not find!!")
    else:
       print(np.squeeze(points_hash[id]))
    time_search.append(end - start)

38.881,52.366,4.492

35.793,40.253,8.69

[ 31.144   8.307 -39.819]
[ 31.144   8.307 -39.819]


In [62]:
time_search

[0.00014901161193847656,
 6.651878356933594e-05,
 0.046727657318115234,
 0.026241779327392578]

#search_method with random data

In [82]:
search_random = []
for model,points in zip(model_trees,point_trees):
    #search point in data
    point = gen_query_point(l_gen,"tree",model.method)
    start = time.time()
    id = kdtree.search(point,model)
    end = time.time()
    if id is None:
       print("not find!!")
    search_random.append(end - start)

for method in method_lsh:
    query_point = gen_query_point(l_gen,"hash")
    start = time.time()
    id = model_hash.check_point_in_data(query_point)
    end = time.time()
    if id == -1:
       print("not find!!")
    else:
       print(np.squeeze(points_hash[id]))
    search_random.append(end - start)

not find!!
not find!!
not find!!
not find!!


In [83]:
print(search_random)

[6.604194641113281e-05, 4.482269287109375e-05, 0.05912899971008301, 0.05121636390686035]


Knn_method

In [84]:
knn_search = []
for model,points in zip(model_trees,point_trees):
    #search point in data
    point = gen_query_point(l_gen,"tree",model.method)
    start = time.time()
    model.findkNN(point,k)
    end = time.time()
    knn_search.append(end - start)

for method in method_lsh:
    query_point = gen_query_point(l_gen,"hash")
    start = time.time()
    model_hash.view_result(method,k,-1,query_point)
    end = time.time()
    knn_search.append(end - start)

point1: 29.187,70.359,87.598 with euclide_distance: 3.9309993640294643

point2: 30.685,68.806,87.511 with euclide_distance: 4.815325326496652

point3: 32.839,72.834,85.092 with euclide_distance: 5.129565381199464

point1: 29.187,70.359,87.598 with euclide_distance: 3.9309993640294643

point2: 30.685,68.806,87.511 with euclide_distance: 4.815325326496652

point3: 32.543,64.282,89.458 with euclide_distance: 8.844414112873737

Number of points to consider:  355
data point to query:  [[31.015 72.983 89.884]]
3 approximate nearest points:

[array([[36.78 , 65.547, 86.775]]),
 array([[21.573, 74.729, 93.254]]),
 array([[27.901, 71.379, 88.603]])]
Number of points to consider:  325
data point to query:  [[31.015 72.983 89.884]]
3 approximate nearest points:

[array([[36.78 , 65.547, 86.775]]),
 array([[21.573, 74.729, 93.254]]),
 array([[27.901, 71.379, 88.603]])]


In [85]:
knn_search

[0.0019497871398925781,
 0.0007760524749755859,
 0.04799604415893555,
 0.05278921127319336]

In [95]:
dic = {'data structure': ['kd-tree not balance', 'kd-tree balance', 'hash_universe', 'hash_flip_method'],
      'time_construct': time_construct,
      'time_search': time_search,
      'search_random' : search_random,
      'knn-search' : knn_search  
      }

In [96]:
df = pd.DataFrame(dic)
print(df)

        data structure  time_construct  time_search  search_random  knn-search
0  kd-tree not balance        0.412248     0.000149       0.000066    0.001950
1      kd-tree balance        0.232631     0.000067       0.000045    0.000776
2        hash_universe        0.807569     0.046728       0.059129    0.047996
3     hash_flip_method        0.807569     0.026242       0.051216    0.052789


In [107]:
def performance_time(l_n,filename,k):
    res = []
    method_tree = ['classic','re_balance']
    method_lsh = ['universe','flipped']

    #for query gen_point
    random.seed(0)
    l_gen = []
    for j in range(0,3):
        g = round(random.uniform(0,100),3)
        l_gen.append(g)
  
    for n in l_n:
        lsh.gen_data(n,filename)
        model_trees = []
        point_trees = []

        #time_construct
        print("construct data structure")
        print("=============================================")
        #====================================================
        time_construct = []
        for method in method_tree:
            start = time.time()
            model,point = kdtree.build_method(filename,method)
            end = time.time()

            model_trees.append(model)
            point_trees.append(point)
            time_construct.append(end - start)

        start = time.time()
        points_hash = lsh.readData(filename)
        model_hash = lsh.LSH(points_hash)
        model_hash.create_planes(10,25)
        model_hash.create_hash_tables()
        end = time.time()
        time_construct.append(end - start)
        time_construct.append(time_construct[-1])
        #====================================================
        print("=============================================")

        print("search_method")
        print("=============================================")
        #search_method:
        index_random = 0
        time_search = []
        for model,points in zip(model_trees,point_trees):
            #search point in data
            point = points[index_random]
            start = time.time()
            kdtree.search(point,model)
            end = time.time()
            time_search.append(end - start)

        for method in method_lsh:
            start = time.time()
            id = model_hash.check_point_in_data(points_hash[index_random])
            end = time.time()
            if id == -1:
              print("not find!!")
            else:
              print(np.squeeze(points_hash[id]))
            time_search.append(end - start)
        print("=============================================")

        print("search_method with random data")
        print("=============================================")
        search_random = []
        for model,points in zip(model_trees,point_trees):
            #search point in data
            point = gen_query_point(l_gen,"tree",model.method)
            start = time.time()
            id = kdtree.search(point,model)
            end = time.time()
            if id is None:
              print("not find!!")
            search_random.append(end - start)

        for method in method_lsh:
            query_point = gen_query_point(l_gen,"hash")
            start = time.time()
            id = model_hash.check_point_in_data(query_point)
            end = time.time()
            if id == -1:
              print("not find!!")
            else:
              print(np.squeeze(points_hash[id]))
            search_random.append(end - start)
        print("=============================================")

        print("knn method")
        print("=============================================")
        knn_search = []
        for model,points in zip(model_trees,point_trees):
            #search point in data
            point = gen_query_point(l_gen,"tree",model.method)
            start = time.time()
            model.findkNN(point,k)
            end = time.time()
            knn_search.append(end - start)

        for method in method_lsh:
            query_point = gen_query_point(l_gen,"hash")
            start = time.time()
            model_hash.view_result(method,k,-1,query_point)
            end = time.time()
            knn_search.append(end - start)
        print("=============================================")

        dic = {'data structure': ['kd-tree not balance', 'kd-tree balance', 'hash_universe', 'hash_flip_method'],
               'time_construct': time_construct,
               'time_search': time_search,
               'search_random' : search_random,
               'knn-search' : knn_search  
              }
        df = pd.DataFrame(dic)
        res.append(df)
    return res

In [108]:
dic = performance_time(n,'a.txt',3)

construct data structure
search_method
68.65,83.463,1.829

28.778,13.003,1.937

[-5.832 -4.097 27.949]
[-5.832 -4.097 27.949]
search_method with random data
not find!!
not find!!
not find!!
not find!!
knn method
point1: 75.014,98.886,74.817 with euclide_distance: 41.17400958128804

point2: 68.65,83.463,1.829 with euclide_distance: 43.891656063538996

point1: 75.014,98.886,74.817 with euclide_distance: 41.17400958128804

point2: 68.65,83.463,1.829 with euclide_distance: 43.891656063538996

point3: 44.789,90.86,29.361 with euclide_distance: 44.277568248493495

Number of points to consider:  379
data point to query:  [[84.442 75.795 42.057]]
3 approximate nearest points:

[array([[95.274, 87.542, 42.783]]),
 array([[80.503, 71.082, 41.033]]),
 array([[57.712, 56.315, 48.655]])]
Number of points to consider:  369
data point to query:  [[84.442 75.795 42.057]]
3 approximate nearest points:

[array([[95.274, 87.542, 42.783]]),
 array([[80.503, 71.082, 41.033]]),
 array([[57.712, 56.315, 48.6

In [115]:
for d,i in zip(dic,n):
    print("data = ",i)
    print(d)

data =  10
        data structure  time_construct  time_search  search_random  knn-search
0  kd-tree not balance        0.002038     0.000411       0.000013    0.000168
1      kd-tree balance        0.001205     0.000264       0.000013    0.000183
2        hash_universe        0.812732     0.037199       0.049695    0.059717
3     hash_flip_method        0.812732     0.048004       0.032468    0.084016
data =  100
        data structure  time_construct  time_search  search_random  knn-search
0  kd-tree not balance        0.003693     0.000232       0.000032    0.000252
1      kd-tree balance        0.008505     0.000249       0.000022    0.000184
2        hash_universe        0.795465     0.028846       0.024303    0.035404
3     hash_flip_method        0.795465     0.024345       0.024813    0.113438
data =  1000
        data structure  time_construct  time_search  search_random  knn-search
0  kd-tree not balance        0.019538     0.000051       0.000045    0.000324
1      kd-tree b

In [124]:
# worksheet.write(dic[0])
name = ['ten_data.csv','one_hundred.csv','one_thousand.csv','ten_thousand.csv','one_hundred_thousand.csv','one_milion.csv']
for i,d in enumerate(dic):
    d.to_csv(name[i])